In [5]:
import pandas as pd

1. 데이터 불러오기

In [16]:
# CSV 파일 불러오기
google_df = pd.read_csv('C:/Users/chica/OneDrive/바탕 화면/BAF-25-1-finance_2/코드/구글_트렌드_병합_스케일링_데이터.csv')
incident_df = pd.read_csv('./전처리/Final_라벨결측제거_원핫인코딩_직업.csv')

2. 날짜 전처리 (YYYY-MM 형태로 변환)

In [17]:
# 사건 날짜를 datetime 형식으로 변환 후 '년월' 컬럼 생성
incident_df['사건 날짜'] = pd.to_datetime(incident_df['사건 날짜'], errors='coerce')
incident_df['년월'] = incident_df['사건 날짜'].dt.to_period('M').astype(str)

# 구글 트렌드의 월도 동일하게 변환
google_df['월'] = pd.to_datetime(google_df['월'], format='%Y-%m')
google_df['년월'] = google_df['월'].dt.to_period('M').astype(str)

3. 관심도 값 병합 함수 정의 및 적용

In [18]:
def get_interest(row):
    date = row['년월']
    name = row['연예인 이름']
    try:
        value = google_df.loc[google_df['년월'] == date, name].values
        if len(value) > 0:
            return value[0]
    except KeyError:
        return None
    return None

incident_df['관심도_스케일링'] = incident_df.apply(get_interest, axis=1)


In [19]:
incident_df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', '소속사', '연예인 이름', '사건 날짜',
       'AR1', 'AR2', '그룹', '사건 내용', '라벨 1', '라벨 2', '라벨 3', '사건 날짜(매핑된 주가 날짜)',
       'CAR(1,2)', '시가총액', '년월', '관심도', 'log_시가총액', '시가총액_랭크',
       'stock_trend_n10', '사생활', '사회적 감수성', '종교/이념', '범죄 혐의', '발언 문제',
       '혐의정보 유포', '세금 문제', '무혐의', '팬 대응', '무의식적 태도', '병역 문제', '기타', '성 관련',
       '직업', '직업_배우', '직업_아이돌', '거래량 변화율', '관심도_스케일링'],
      dtype='object')

In [21]:
incident_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,소속사,연예인 이름,사건 날짜,AR1,AR2,그룹,사건 내용,...,팬 대응,무의식적 태도,병역 문제,기타,성 관련,직업,직업_배우,직업_아이돌,거래량 변화율,관심도_스케일링
0,0,0,0,HYBE,지민,2022-04-25,0.027321,0.018462,방탄소년단,"2022년 4월 25일, 언론사 비즈한국의 취재결과 건강보험료 체납으로 보유하고 있...",...,1,0,0,0,0,아이돌,False,True,-0.244560,0.0
1,1,1,1,HYBE,정국,2024-01-06,-0.017447,0.018590,방탄소년단,"2024년 1월 6일, 정국이 입대 후 신병 교육을 받는 동안 발생했다. 탈취범은 ...",...,0,0,0,1,0,아이돌,False,True,-0.550694,0.0
2,2,2,2,HYBE,제이,2023-01-10,-0.001733,-0.012679,엔하이픈,"2023년 1월 10일, 멤버 성훈과 함께 진행한 위버스 라이브 도중 요즘 한국사 ...",...,0,1,0,0,0,아이돌,False,True,-0.127025,0.5
3,3,3,3,HYBE,김채원,2023-09-05,-0.003702,-0.011398,르세라핌,2023년 9월 5일 방송통신심의위원회 방송소위원회에서 “경찰에서 무혐의 결론이 났...,...,0,0,0,0,0,아이돌,False,True,-0.229927,0.5
4,4,4,4,HYBE,김채원,2022-10-19,-0.025109,0.010665,르세라핌,2022년 10월 19일 김채원 과 허윤진 이 차량 이동 중 접촉 사고 를 당했다....,...,0,0,0,1,0,아이돌,False,True,-0.575915,0.5


관심도 '<1' 처리 및 숫자형 변환

In [22]:
# '<1' 값을 0.5로, 나머지는 숫자형으로 변환
def clean_interest(val):
    if val == '<1':
        return 0.5
    try:
        return float(val)
    except:
        return None

incident_df['관심도_스케일링'] = incident_df['관심도_스케일링'].apply(clean_interest)

결과 확인

In [23]:
# 결과 미리보기
incident_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,소속사,연예인 이름,사건 날짜,AR1,AR2,그룹,사건 내용,...,팬 대응,무의식적 태도,병역 문제,기타,성 관련,직업,직업_배우,직업_아이돌,거래량 변화율,관심도_스케일링
0,0,0,0,HYBE,지민,2022-04-25,0.027321,0.018462,방탄소년단,"2022년 4월 25일, 언론사 비즈한국의 취재결과 건강보험료 체납으로 보유하고 있...",...,1,0,0,0,0,아이돌,False,True,-0.244560,0.0
1,1,1,1,HYBE,정국,2024-01-06,-0.017447,0.018590,방탄소년단,"2024년 1월 6일, 정국이 입대 후 신병 교육을 받는 동안 발생했다. 탈취범은 ...",...,0,0,0,1,0,아이돌,False,True,-0.550694,0.0
2,2,2,2,HYBE,제이,2023-01-10,-0.001733,-0.012679,엔하이픈,"2023년 1월 10일, 멤버 성훈과 함께 진행한 위버스 라이브 도중 요즘 한국사 ...",...,0,1,0,0,0,아이돌,False,True,-0.127025,0.5
3,3,3,3,HYBE,김채원,2023-09-05,-0.003702,-0.011398,르세라핌,2023년 9월 5일 방송통신심의위원회 방송소위원회에서 “경찰에서 무혐의 결론이 났...,...,0,0,0,0,0,아이돌,False,True,-0.229927,0.5
4,4,4,4,HYBE,김채원,2022-10-19,-0.025109,0.010665,르세라핌,2022년 10월 19일 김채원 과 허윤진 이 차량 이동 중 접촉 사고 를 당했다....,...,0,0,0,1,0,아이돌,False,True,-0.575915,0.5


In [24]:
incident_df = incident_df.drop(columns=['년월'])
incident_df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,소속사,연예인 이름,사건 날짜,AR1,AR2,그룹,사건 내용,...,팬 대응,무의식적 태도,병역 문제,기타,성 관련,직업,직업_배우,직업_아이돌,거래량 변화율,관심도_스케일링
0,0,0,0,HYBE,지민,2022-04-25,0.027321,0.018462,방탄소년단,"2022년 4월 25일, 언론사 비즈한국의 취재결과 건강보험료 체납으로 보유하고 있...",...,1,0,0,0,0,아이돌,False,True,-0.244560,0.0
1,1,1,1,HYBE,정국,2024-01-06,-0.017447,0.018590,방탄소년단,"2024년 1월 6일, 정국이 입대 후 신병 교육을 받는 동안 발생했다. 탈취범은 ...",...,0,0,0,1,0,아이돌,False,True,-0.550694,0.0
2,2,2,2,HYBE,제이,2023-01-10,-0.001733,-0.012679,엔하이픈,"2023년 1월 10일, 멤버 성훈과 함께 진행한 위버스 라이브 도중 요즘 한국사 ...",...,0,1,0,0,0,아이돌,False,True,-0.127025,0.5
3,3,3,3,HYBE,김채원,2023-09-05,-0.003702,-0.011398,르세라핌,2023년 9월 5일 방송통신심의위원회 방송소위원회에서 “경찰에서 무혐의 결론이 났...,...,0,0,0,0,0,아이돌,False,True,-0.229927,0.5
4,4,4,4,HYBE,김채원,2022-10-19,-0.025109,0.010665,르세라핌,2022년 10월 19일 김채원 과 허윤진 이 차량 이동 중 접촉 사고 를 당했다....,...,0,0,0,1,0,아이돌,False,True,-0.575915,0.5


In [25]:
# CSV 저장
incident_df.to_csv('./전처리/Final_라벨결측제거_원핫인코딩_직업.csv', encoding='utf-8-sig', index=False)